# AlexNet
이 노트북은 AlexNet을 구현하는 노트북이다.


In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
# 데이터셋 불러오기
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# 이름 설정하기
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# 데이터 split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# 결과 확인
print("x_train shape: ", x_train.shape)
print("y_train shape: ", y_train.shape)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 12s 0us/step
x_train shape:  (40000, 32, 32, 3)
y_train shape:  (40000, 1)


In [3]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
val_ds = tf.data.Dataset.from_tensor_slices((x_val, y_val))

print("train_ds: ", train_ds)
print("test_ds: ", test_ds)
print("val_ds: ", val_ds)

train_ds:  <_TensorSliceDataset element_spec=(TensorSpec(shape=(32, 32, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(1,), dtype=tf.uint8, name=None))>
test_ds:  <_TensorSliceDataset element_spec=(TensorSpec(shape=(32, 32, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(1,), dtype=tf.uint8, name=None))>
val_ds:  <_TensorSliceDataset element_spec=(TensorSpec(shape=(32, 32, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(1,), dtype=tf.uint8, name=None))>


In [4]:
def process_images(image, label):

    # Resize images form 32*32 to 227*227
    image = tf.image.resize(image, (227, 227))
    return image, label

In [5]:
train_ds = train_ds.batch(128, drop_remainder=True)
test_ds = test_ds.batch(128, drop_remainder=True)
val_ds = val_ds.batch(128, drop_remainder=True)

train_ds = train_ds.map(process_images)
test_ds = test_ds.map(process_images)
val_ds = val_ds.map(process_images)

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization, MaxPool2D

class AlexNet(Sequential):
    def __init__(self):
        super().__init__()

        self.add(Conv2D(96, kernel_size=11, strides=4, activation='relu', input_shape=(227, 227, 3)))
        self.add(BatchNormalization())
        self.add(MaxPool2D(3, 2))

        self.add(Conv2D(256, kernel_size=5, strides=1, activation='relu', padding='same'))
        self.add(BatchNormalization())
        self.add(MaxPool2D(3, 2))

        self.add(Conv2D(384, kernel_size=3, strides=1, activation='relu', padding='same'))
        self.add(BatchNormalization())

        self.add(Conv2D(384, kernel_size=3, strides=1, activation='relu', padding='same'))
        self.add(BatchNormalization())

        self.add(Conv2D(256, kernel_size=3, strides=1, activation='relu', padding='same'))
        self.add(BatchNormalization())
        self.add(MaxPool2D(3, 2))

        self.add(Flatten())
        self.add(Dense(4096, activation='relu'))
        self.add(Dropout(0.5))
        self.add(Dense(4096, activation='relu'))
        self.add(Dropout(0.5))
        self.add(Dense(10, activation='softmax'))

        self.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [7]:
model = AlexNet()

model.summary()

c:\Users\jkwor\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "alex_net"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 55, 55, 96)     │        34,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 55, 55, 96)     │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 27, 27, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 27, 27, 256)    │       614,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 27, 27, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 13, 13, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 13, 13, 384)    │       885,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 13, 13, 384)    │         1,536 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 13, 13, 384)    │     1,327,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 13, 13, 384)    │         1,536 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 13, 13, 256)    │       884,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 13, 13, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4096)           │    37,752,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │        40,970 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 58,327,818 (222.50 MB)

 Trainable params: 58,325,066 (222.49 MB)

 Non-trainable params: 2,752 (10.75 KB)

In [9]:
model.fit(train_ds, validation_data=val_ds, epochs=100)

Epoch 1/100
312/312 ━━━━━━━━━━━━━━━━━━━━ 737s 2s/step - accuracy: 0.7253 - loss: 0.8201 - val_accuracy: 0.6965 - val_loss: 0.9090
Epoch 2/100
312/312 ━━━━━━━━━━━━━━━━━━━━ 736s 2s/step - accuracy: 0.7501 - loss: 0.7497 - val_accuracy: 0.7355 - val_loss: 0.7996
Epoch 3/100
312/312 ━━━━━━━━━━━━━━━━━━━━ 735s 2s/step - accuracy: 0.7687 - loss: 0.6866 - val_accuracy: 0.7121 - val_loss: 0.8638
Epoch 4/100
312/312 ━━━━━━━━━━━━━━━━━━━━ 736s 2s/step - accuracy: 0.7894 - loss: 0.6239 - val_accuracy: 0.6926 - val_loss: 0.9336
Epoch 5/100
312/312 ━━━━━━━━━━━━━━━━━━━━ 738s 2s/step - accuracy: 0.8076 - loss: 0.5731 - val_accuracy: 0.7588 - val_loss: 0.7339
Epoch 6/100
312/312 ━━━━━━━━━━━━━━━━━━━━ 739s 2s/step - accuracy: 0.8284 - loss: 0.5259 - val_accuracy: 0.7464 - val_loss: 0.7936
Epoch 7/100
312/312 ━━━━━━━━━━━━━━━━━━━━ 741s 2s/step - accuracy: 0.8425 - loss: 0.4757 - val_accuracy: 0.7428 - val_loss: 0.8181
Epoch 8/100
312/312 ━━━━━━━━━━━━━━━━━━━━ 739s 2s/step - accuracy: 0.8543 - loss: 0.4502 - 